In [1]:
import numpy as np
import calendar
import requests
import operator
from datetime import datetime
import copy
import pandas as pd
from operator import itemgetter
import json

In [48]:
import cla_formatter

ModuleNotFoundError: No module named 'cla_formatter'

In [539]:
max_pres = [
                30.45,
                30.61,
                30.53,
                "M",
                "M",
                30.21,
                30.17,
                30.14,
                "M",
                30.48,
                30.53,
                30.65
            ]

max_pres_dates = [
                "JAN 31",
                "FEB 3",
                "MAR 19",
                "M",
                "M",
                "JUN 20",
                "JUL 31",
                "AUG 1",
                "M",
                "OCT 31",
                "NOV 1",
                "DEC 14"
            ]

In [547]:
def _get_maxmin(val_lst, dt_lst):

    _temp_lst = [val for val in val_lst if val != 'M']
    idx = val_lst.index(np.max(_temp_lst))
    dt = dt_lst[idx]
    return np.max(_temp_lst), dt

In [552]:
x = [1]
y = [2]

if x and y:
    print('yep')

yep


In [553]:
data_dict = {"wind": {
            "max_gst": [
                50,
                53,
                50,
                54
            ],
            "max_wdr": [
                190,
                170,
                240,
                330
            ],
            "max_wd_dates": [
                "2023-02-26",
                "2023-02-27",
                "2023-03-31",
                "2023-07-30"
            ]
        }
            }

data_dict['wind']

{'max_gst': [50, 53, 50, 54],
 'max_wdr': [190, 170, 240, 330],
 'max_wd_dates': ['2023-02-26', '2023-02-27', '2023-03-31', '2023-07-30']}

In [562]:
#def _format_wind():

wsp = data_dict['wind']['max_gst']
wdr = data_dict['wind']['max_wdr']
dates = data_dict['wind']['max_wd_dates']

def _format_windgusts(wsp, wdr, dates):
    
    # format the date
    m = [datetime.strptime(d, '%Y-%m-%d').strftime('%B').upper() for d in dates]
    d = [_find_numeric_suffix(d.split('-')[-1]) for d in dates]
    
    dts = [f'{m} {d}' for m,d in zip(m,d)]
    
    for wsp, wdr, dts in zip(wsp, wdr, dts):
        f.write(f'WINDS GUSTED TO {wsp} MPH/{wdr} DEGREES ON {dts}.\n\n')

if wsp:
    _format_windgusts(wsp, wdr, dates)




ValueError: I/O operation on closed file.

In [548]:
max_slp, max_slp_dt = _get_maxmin(max_pres, max_pres_dates)
max_slp, max_slp_dt

(30.65, 'DEC 14')

In [7]:
'''
Here we were working with the xmacis API to obtain data, we have working code to get annual avg data and departure from normal,
now we need some logic to compare a yearly value 

'''


def get_annual_xmacis_data(station):

    '''
    Data comes out as follows, a lists of lists formatted as such:
    
    [yr, [[jan avg max T, msng cnt],[feb avg max T, msng cnt], etc...], 
         [[jan avg min T, msng cnt],[feb avg min T, msng cnt], etc...], 
         [[jan avg mean T, msng cnt],[feb avg mean T, msng cnt], etc...], 
         [[jan total precip, msng cnt], [feb total precip, msng cnt], etc...],
         [[jan total snow, msng cnt], [feb total snow, msng cnt], etc...],
         ]
    
    '''

    
    # a dictionary that contains the json syntax to the xmacis api for the four climate sites
    json_dict = {
        'lit' : '{"sid":"LITthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year","prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":2},{"name":"snow","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":1}]}',
        'lzk' : '{"sid":"LZK","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year","prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":2},{"name":"snow","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":1}]}',
        'pbf' : '{"sid":"PBFthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year","prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":2},{"name":"snow","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":1}]}',
        'hro' : '{"sid":"HROthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"mean"},"maxmissing":"7","groupby":"year","prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":2},{"name":"snow","interval":[0,1],"duration":1,"reduce":{"add":"mcnt","reduce":"sum"},"maxmissing":"7","groupby":"year","prec":1}]}'

        # old format not being used anymore
        # 'lit' : '{"sid":"LITthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year", "prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","prec":2,"groupby":"year"},{"name":"snow","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","groupby":"year","prec":1}],"output":"json"}',
        # 'lzk' : '{"sid":"LZKthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year", "prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","prec":2,"groupby":"year"},{"name":"snow","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","groupby":"year","prec":1}],"output":"json"}',
        # 'pbf' : '{"sid":"PBFthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year", "prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","prec":2,"groupby":"year"},{"name":"snow","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","groupby":"year","prec":1}],"output":"json"}',
        # 'hro' : '{"sid":"HROthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"mint","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year"},{"name":"avgt","interval":[0,1],"duration":1,"reduce":"mean","maxmissing":"7","groupby":"year", "prec":0},{"name":"pcpn","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","prec":2,"groupby":"year"},{"name":"snow","interval":[0,1],"duration":1,"reduce":"sum","maxmissing":"7","groupby":"year","prec":1}],"output":"json"}',

    }

    json_text = json_dict[station.lower()]
    
    # the xmacis api server for station data
    url = f"https://data.rcc-acis.org/StnData?params={json_text}"
    
    # A GET request to the API
    r = requests.get(url)
    return r.json()

#main_data = get_annual_xmacis_data('HRO')



In [31]:
year = 2022

main_data = get_annual_xmacis_data('LZK')

# drop the current year (last list in the array) if it is the current year
# technically, this shouldn't be run in the current year anyways, only for troubleshooting...
if year != datetime.now().year:
    #data = main_data['data'][:-1]
    diff = year - datetime.now().year
    data = main_data['data'][:diff]

else:
    data = main_data['data']

# string for the period of record to be included in the final data dictionary
por = [data[0][0],data[-1][0], int(data[-1][0]) - int(data[0][0])]

por

-2


['1975', '2022', 47]

In [28]:
data

[['1975',
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]]],
 ['1976',
  [['50.5', 0],
   ['64.2', 0],
   ['66.8', 0],
   ['72.2', 0],
   ['74.1', 0],
   ['83.2', 0],
   ['90.6', 0],
   ['89.1', 0],
   ['80.3', 0],
   ['67.9', 0],
   ['55.0', 0],
   ['51.2', 0]],
  [['29.6', 0],
   ['41.9', 0],
   ['45.5', 0],
   ['52.6', 0],
   ['55.4', 0],
   ['65.2', 0],
   ['70.9', 0],
   ['67.6', 0],
   ['62.4', 0],
   ['46.9', 0],
   ['34.5', 0],
   ['30.6', 0]],
  [['40.0', 0],
   ['53.1', 0],
   ['56.2', 0],
   ['62.4', 0],
   ['64.7', 0],
   ['74.2', 0],
   ['80.7', 0],
   ['78.3', 0],
   ['71.3', 0],
   ['57.4', 0],
   ['44.7', 0],
   ['40.9', 0]],
  [['2.48', 0],
   ['2.80', 0],
   ['5.66', 0],
  

In [10]:
# lets import normals data

nrml_file = pd.read_csv('./normals/LZK_Normals.csv')
nrml_df = pd.DataFrame(nrml_file)

In [24]:

# these are the current working functions - 8/23/2024

def _sort_annual_temp_data(data, key, index):

    # strip the years
    yrs = [int(lst[0]) for lst in data]
    
    # strip the yearly data for a parameter
    yrly_data = [lst[index] for lst in data]
    
    # strip the monthly parameter data
    mnly_data = [[mnly_lst[0] for mnly_lst in yrly_lst] for yrly_lst in yrly_data] # this parses the monthly values out into a list of lists
    
    # calculate the total number of missing days for each year
    # had to add an exception here for mnly_list, mainly for LZK, where in the first year, data was not available until 12/31/1975,
    # so all other months for 1975 were nan, but not in format [monthly val, missing days], rather just monthly val
    msng_cnt = [np.sum([mnly_lst[1] if isinstance(mnly_lst, list) else 0 for mnly_lst in yrly_lst]) for yrly_lst in yrly_data]
    
    # convert missing to nan's
    param = [[val if val != 'M' else np.nan for val in lst] for lst in mnly_data]
    
    # this will only work if 'M' values have been converted to nans, will need to address precip 'T' values as well...
    param = [[np.float64(val) for val in lst] for lst in param]
    
    # calculate the mean value of the paramter (either maxt, mint, or avgt)
    _param = []
    
    for lst in param:
        if not np.isnan(lst).all():
            _param.append(np.round(np.nanmean(lst),1))

        else:
            _param.append(np.nan)
    #param = [np.round(np.nanmean(lst),1) for lst in param]

    # convert _param back to param
    param = _param
    
    # now lets calculate the dfn's
    dfn = [np.round(val - nrml_df[key].values[-1], 1) if not np.isnan(val) else np.nan for val in param]
    
    # now we need to put the data back together...
    # in the form of [yr, val, dfn, msng_count]
    yr_annparam_dfn = [[y, val, dfn, msng_ct] for y, val, dfn, msng_ct in zip(yrs, param, dfn, msng_cnt)]
    
    # this will remove years where the number of missing days exceeds 7
    yr_annparam_dfn_filtered = [lst for lst in yr_annparam_dfn if lst[-1] <= 7]

    # get the number of excluded years
    num_excluded_yrs = len(yrs) - len(yr_annparam_dfn_filtered)
    
    # this will create a sorted list, with the largest values first
    # note, in itemgetter, the tuple (1,0) will sort by the paramter value, and then sort by year, placing the most recent year ahead
    return sorted(yr_annparam_dfn_filtered, key=itemgetter(1,0), reverse = True), str(num_excluded_yrs)




def _sort_annual_precip_data(data, key, index, prec):

    # strip the years
    yrs = [int(lst[0]) for lst in data]
    
    # strip the yearly data for a parameter
    yrly_data = [lst[index] for lst in data]
    
    # strip the monthly parameter data
    mnly_data = [[mnly_lst[0] for mnly_lst in yrly_lst] for yrly_lst in yrly_data] # this parses the monthly values out into a list of lists
    
    # # calculate the total number of missing days for each year
    # had to add an exception here for mnly_list, mainly for LZK, where in the first year, data was not available until 12/31/1975,
    # so all other months for 1975 were nan, but not in format [monthly val, missing days], rather just monthly val
    msng_cnt = [np.sum([mnly_lst[1] if isinstance(mnly_lst, list) else 0 for mnly_lst in yrly_lst]) for yrly_lst in yrly_data]
    
    # convert missing to nan's
    param = [[val if val != 'M' else np.nan for val in lst] for lst in mnly_data]
    
    # # this will only work if 'M' values have been converted to nans, will need to address precip 'T' values as well...
    param = [[np.float64(val) if val != 'T' else val for val in lst] for lst in param]

    
    # iterate through the snowfall data to handle trace values, there will be numerous cases of trace values...
    _param_filtered = []
    
    for lst in param:
        # if there are no trace values, then calculate sum as normal
        if 'T' not in lst:
            
            if not np.isnan(lst).all():
                _param_filtered.append(np.round(np.nansum(lst), prec))   
                
            # add a nan value if all monthly values for the year are nan
            else:
                _param_filtered.append(np.nan)
        
        # if there are trace values in the list, then filter them out
        else:
            filtered_lst = [val if val != 'T' else 0.00001 for val in lst]
            
            # if the sum is < 0.1, then annual snowfall was only trace 'T'
            if np.nansum(filtered_lst) < 0.1:
                _param_filtered.append(0.00001)
    
            # if the sum is >= 0.1 after filtering out trace values, find the sum
            elif np.nansum(filtered_lst) >= 0.1:
                _param_filtered.append(np.round(np.nansum(filtered_lst), prec))
        
        # reclassify the temporary list as the main list again
        param_filtered = _param_filtered
        
        # now lets calculate the dfn's, handle trace values, else just normal*-1
        dfn = [np.round(val - nrml_df[key].values[-1], prec) if val != 0.00001 or val is not np.isnan(val) else nrml_df['SNOW'].values[-1]*-1 for val in param_filtered] 

    
    # now we need to put the data back together...
    # in the form of [yr, val, dfn, msng_count]
    yr_annparam_dfn = [[y, val, dfn, msng_ct] for y, val, dfn, msng_ct in zip(yrs, param_filtered, dfn, msng_cnt)]
    
    #print(yr_annparam_dfn)
    
    # this will remove years where the number of missing days exceeds 7
    yr_annparam_dfn_filtered = [lst for lst in yr_annparam_dfn if lst[-1] <= 7]

    # get the number of excluded years
    num_excluded_yrs = len(yrs) - len(yr_annparam_dfn_filtered)
    
    # this will create a sorted list, with the largest values first
    # note, in itemgetter, the tuple (1,0) will sort by the paramter value, and then sort by year, placing the most recent year ahead
    return sorted(yr_annparam_dfn_filtered, key=itemgetter(1,0), reverse = True), str(num_excluded_yrs)
    



In [13]:
data

[['1975',
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]],
  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', ['M', 30]]],
 ['1976',
  [['50.5', 0],
   ['64.2', 0],
   ['66.8', 0],
   ['72.2', 0],
   ['74.1', 0],
   ['83.2', 0],
   ['90.6', 0],
   ['89.1', 0],
   ['80.3', 0],
   ['67.9', 0],
   ['55.0', 0],
   ['51.2', 0]],
  [['29.6', 0],
   ['41.9', 0],
   ['45.5', 0],
   ['52.6', 0],
   ['55.4', 0],
   ['65.2', 0],
   ['70.9', 0],
   ['67.6', 0],
   ['62.4', 0],
   ['46.9', 0],
   ['34.5', 0],
   ['30.6', 0]],
  [['40.0', 0],
   ['53.1', 0],
   ['56.2', 0],
   ['62.4', 0],
   ['64.7', 0],
   ['74.2', 0],
   ['80.7', 0],
   ['78.3', 0],
   ['71.3', 0],
   ['57.4', 0],
   ['44.7', 0],
   ['40.9', 0]],
  [['2.48', 0],
   ['2.80', 0],
   ['5.66', 0],
  

In [25]:
sorted_maxt, n1 = _sort_annual_temp_data(data, key = 'MAX', index = 1)
sorted_mint, n2 = _sort_annual_temp_data(data, key = 'MIN', index = 2)
sorted_avgt, n3 = _sort_annual_temp_data(data, key = 'MEAN', index = 3)

sorted_pcpn, n4 = _sort_annual_precip_data(data, key = 'PCPN', index = 4, prec = 2)
sorted_snow, n5 = _sort_annual_precip_data(data, key = 'SNOW', index = 5, prec = 1)

[[2009, 79.61, 27.76, 0],
 [2018, 70.54, 18.69, 0],
 [2015, 67.06, 15.21, 0],
 [2019, 66.11, 14.26, 0],
 [1990, 63.65, 11.8, 0],
 [1982, 62.99, 11.14, 0],
 [2011, 62.98, 11.13, 0],
 [2020, 60.28, 8.43, 0],
 [1984, 59.41, 7.56, 0],
 [1997, 59.21, 7.36, 0],
 [1991, 58.45, 6.6, 0],
 [2004, 57.79, 5.94, 0],
 [1978, 56.24, 4.39, 0],
 [2023, 55.02, 3.17, 0],
 [1987, 54.96, 3.11, 0],
 [1994, 54.95, 3.1, 0],
 [1979, 52.58, 0.73, 0],
 [2016, 52.46, 0.61, 0],
 [2008, 51.93, 0.08, 0],
 [2021, 51.76, -0.09, 0],
 [2017, 51.48, -0.37, 0],
 [1996, 51.43, -0.42, 0],
 [2013, 51.3, -0.55, 0],
 [1985, 50.93, -0.92, 0],
 [2001, 50.26, -1.59, 0],
 [2006, 49.78, -2.07, 0],
 [2007, 49.05, -2.8, 0],
 [1989, 48.87, -2.98, 0],
 [1988, 47.94, -3.91, 0],
 [1993, 47.69, -4.16, 0],
 [2002, 47.42, -4.43, 0],
 [2014, 47.15, -4.7, 0],
 [1992, 46.05, -5.8, 0],
 [1977, 45.08, -6.77, 0],
 [2022, 44.98, -6.87, 1],
 [2012, 44.33, -7.52, 0],
 [1986, 43.86, -7.99, 0],
 [2000, 43.56, -8.29, 0],
 [1983, 42.57, -9.28, 0],
 [201

In [438]:
# now lets assemble all the data into a dictionary, to be exported to a text file as json text
_sorted_data = {
    'sorted_avg_maxt' : [[str(val) for val in subl] for subl in sorted_maxt],
    'no_yrs_excluded_avg_maxt' : n1,
    
    'sorted_avg_mint' : [[str(val) for val in subl] for subl in sorted_mint],
    'no_yrs_excluded_avg_mint' : n2,    
    
    'sorted_avg_meant' : [[str(val) for val in subl] for subl in sorted_avgt],
    'no_yrs_excluded_avg_meant' : n3,  
    
    'sorted_total_pcpn' : [[str(val) for val in subl] for subl in sorted_pcpn],
    'no_yrs_excluded_total_pcpn' : n4,  
    
    'sorted_total_snow' : [[str(val) for val in subl] for subl in sorted_snow],
    'no_yrs_excluded_total_snow' : n5,  
    
    }

_meta_data = {
    'site_name' : main_data['meta']['name'],
    'period_of_record' : por,
    'syntax' : '[year, avg or total value, departure from normal, no. missing days for year]'
}

# declare the new dictionary
xmacis_sorted_data = {}

# append the sub-dictionaries
xmacis_sorted_data['meta'] = _meta_data
xmacis_sorted_data['data'] = _sorted_data



In [439]:
xmacis_sorted_data

{'meta': {'site_name': 'Harrison Area',
  'period_of_record': ['1892', '2023', 131],
  'syntax': '[year, avg or total value, departure from normal, no. missing days for year]'},
 'data': {'sorted_avg_maxt': [['1954', '74.7', '5.7', '1'],
   ['2012', '73.2', '4.2', '0'],
   ['1901', '73.1', '4.1', '7'],
   ['1956', '72.6', '3.6', '0'],
   ['1952', '72.6', '3.6', '2'],
   ['1934', '72.6', '3.6', '7'],
   ['1964', '72.0', '3.0', '0'],
   ['1909', '71.9', '2.9', '2'],
   ['1941', '71.8', '2.8', '6'],
   ['1955', '71.5', '2.5', '2'],
   ['1998', '71.1', '2.1', '2'],
   ['2017', '71.0', '2.0', '0'],
   ['2006', '71.0', '2.0', '0'],
   ['1962', '71.0', '2.0', '0'],
   ['2023', '70.9', '1.9', '0'],
   ['2016', '70.8', '1.8', '0'],
   ['1999', '70.7', '1.7', '0'],
   ['1959', '70.7', '1.7', '2'],
   ['1957', '70.5', '1.5', '2'],
   ['1974', '70.3', '1.3', '2'],
   ['2005', '70.2', '1.2', '0'],
   ['1990', '70.2', '1.2', '0'],
   ['1914', '70.2', '1.2', '3'],
   ['1904', '70.2', '1.2', '7'],
   

In [440]:
pil = 'CLMHRO'

with open(f'./output/XMACIS_SortedData_{pil[3:]}.txt', 'w') as f:
    f.write(json.dumps(xmacis_sorted_data, indent = 4))

In [ ]:
'''
Everything above works, now we need to get to formatting the text statements to add to the final text file...

'''

In [499]:
def _find_numeric_suffix(myDate):
    '''
    This function will take a string date, formatted as 'nn', e.g. '05', and assign a suffix based on the number.

    Parameters
    ----------
    myDate : str, a date number, formatted as 'nn', e.g. '05'

    Returns
    ----------
    myDate : str, formatted as 'nnTH', 'nnST', 'nnND', 'nnRD'

    '''

    date_suffix = ["TH", "ST", "ND", "RD"]

    if int(myDate) % 10 in [1, 2, 3] and int(myDate) not in [11, 12, 13]:
        return f'{myDate}{date_suffix[int(myDate) % 10]}'
    else:
        return f'{myDate}{date_suffix[0]}'

In [537]:


def _write_rank_text(lst, key, year):

    # get the current year value and dfn for a paramter
    yrly_val = [val[1] for val in lst if val[0] == year][0]
    yrly_dfn = [val[2] for val in lst if val[0] == year][0] 
    
    # logic to determine if the annual value is a tied value
    tie_bool = False
    n = 0
    for sublst in lst:
        if yrly_val in sublst:
            n+=1
    if n > 1:
        tie_bool = True
    
    # here's the logic for ranking a value amongst the sorted years...
    rank = 1
    for idx, sublst in enumerate(lst):
    
        # check if our yearly value matches the value in the rank
        # if it does not, then keep counting the rank, if it does, then its a tie and don't count it
        if sublst[1] != yrly_val:
            rank = rank + 1
    
        # if its our search year, then break the loop
        if sublst[0] == year:
            break

    # check the tie_bool variable, if we need to change the value of idx to incorporate the tied values
    if tie_bool:
        for i, sublst in enumerate(reversed(lst)):
            if yrly_val in sublst:
                idx = len(lst) - i  # return the index in the original list
                break
            
    # if the departure from normal is >= 0.0(0), cut to the top half of the dataset
    if yrly_dfn >= 0.0:
        yrly_dfn = f'+{yrly_dfn}' # format the dfn to include '+'
        filtered_data = lst[:idx]
    
    # if the departure from normal is < 0.0(0), cut to the lower half of the dataset
    elif yrly_dfn < 0.0:
        filtered_data = lst[idx:]

    filtered_data = [lst for lst in filtered_data if lst[0] != year] # check and remove if the list has the current year
    
    # calculate residules over the filtered list to find the min residual and thus, the next closest year
    res = []
    for sublst in filtered_data:
        res.append(abs(year - sublst[0]))
    
    next_closest_year = filtered_data[res.index(min(res))][0]

    # words to fill into the default text strings
    replacement_words = [f'{yrly_val}', f'{yrly_dfn}', f'{_find_numeric_suffix(rank)}', f'{next_closest_year}']

    # if dfn >= 0.0, use 'warmest', 'wettest', 'snowiest' terminology
    if float(yrly_dfn) >= 0.0:
        i = 0 # default statement
        # if it is a tie, use 'tied' terminology
        if tie_bool:
            i = 1
        _text = text_dict[key.lower()][i].format(*replacement_words).upper()

    # if dfn < 0.0, use 'coldest', 'driest', 'least snowiest' terminology
    elif float(yrly_dfn) < 0.0:
        i = 2 # default statement
        if tie_bool:
            # if it is a tie, use 'tied' terminology
            i = 3
        _text = text_dict[key.lower()][i].format(*replacement_words).upper()
        
    return _text


In [533]:
yrly_val = 60.0

for i, sublst in enumerate(reversed(sorted_avgt)):
    print(sublst)
    if yrly_val in sublst:
        idx = len(sorted_avgt) - i - 1  # return the index in the original list
        break
print(idx)

[1910, 54.9, -3.4, 3]
[1979, 55.1, -3.2, 0]
[1903, 55.2, -3.1, 6]
[1906, 55.8, -2.5, 2]
[1968, 55.9, -2.4, 2]
[1911, 56.1, -2.2, 2]
[1978, 56.1, -2.2, 0]
[1917, 56.2, -2.1, 3]
[1976, 56.2, -2.1, 0]
[1989, 56.3, -2.0, 0]
[2014, 56.3, -2.0, 0]
[1993, 56.4, -1.9, 0]
[1969, 56.6, -1.7, 3]
[1983, 56.6, -1.7, 4]
[2008, 56.7, -1.6, 0]
[1958, 56.8, -1.5, 1]
[2013, 56.9, -1.4, 0]
[1902, 57.0, -1.3, 5]
[1996, 57.1, -1.2, 0]
[1966, 57.2, -1.1, 4]
[2009, 57.2, -1.1, 4]
[1975, 57.3, -1.0, 1]
[1985, 57.3, -1.0, 2]
[2003, 57.3, -1.0, 0]
[1961, 57.4, -0.9, 6]
[1960, 57.5, -0.8, 6]
[2002, 57.5, -0.8, 0]
[1916, 57.6, -0.7, 6]
[1974, 57.6, -0.7, 3]
[1997, 57.6, -0.7, 0]
[1967, 57.7, -0.6, 2]
[1981, 57.7, -0.6, 0]
[1988, 57.7, -0.6, 0]
[1992, 57.7, -0.6, 0]
[1959, 58.0, -0.3, 2]
[1955, 58.1, -0.2, 4]
[1977, 58.1, -0.2, 0]
[1984, 58.1, -0.2, 2]
[2004, 58.1, -0.2, 0]
[1957, 58.2, -0.1, 5]
[2000, 58.2, -0.1, 0]
[1995, 58.3, 0.0, 1]
[2001, 58.3, 0.0, 0]
[1973, 58.4, 0.1, 3]
[1980, 58.4, 0.1, 0]
[1994, 58.4, 0

In [534]:
sorted_avgt[:(idx+1)]

[[2012, 61.7, 3.4, 0],
 [1998, 61.3, 3.0, 4],
 [1954, 61.2, 2.9, 2],
 [1934, 60.2, 1.9, 7],
 [2016, 60.1, 1.8, 0],
 [1999, 60.1, 1.8, 0],
 [2023, 60.0, 1.7, 0],
 [2017, 60.0, 1.7, 0],
 [2006, 60.0, 1.7, 0],
 [1941, 60.0, 1.7, 7]]

In [538]:
sorted_data = [
    sorted_maxt,
    sorted_mint,
    sorted_avgt,
    sorted_pcpn,
    sorted_snow]

for lst, key in zip(sorted_data, text_dict.keys()):
    _get_rank(lst, key, year)


THE YEARLY AVERAGE HIGH TEMPERATURE OF 70.9 (+1.9) WAS THE 15TH WARMEST YEAR ON RECORD, AND THE WARMEST SINCE 2017.
THE YEARLY AVERAGE LOW TEMPERATURE OF 49.1 (+1.5) WAS TIED FOR THE 7TH WARMEST YEAR ON RECORD, AND THE WARMEST SINCE 2017.
THE YEARLY AVERAGE MEAN TEMPERATURE OF 60.0 (+1.7) WAS TIED FOR THE 7TH WARMEST YEAR ON RECORD, AND THE WARMEST SINCE 2017.
THE YEARLY RAINFALL TOTAL OF 44.64 INCHES (+0.14) WAS THE 37TH WETTEST YEAR ON RECORD AND THE WETTEST SINCE 2021.
THE YEARLY SNOWFALL TOTAL OF 11.0 INCHES (+1.5) WAS THE 29TH SNOWIEST YEAR ON RECORD AND THE SNOWIEST SINCE 2022.


In [520]:
sorted_avgt

[[2012, 61.7, 3.4, 0],
 [1998, 61.3, 3.0, 4],
 [1954, 61.2, 2.9, 2],
 [1934, 60.2, 1.9, 7],
 [2016, 60.1, 1.8, 0],
 [1999, 60.1, 1.8, 0],
 [2023, 60.0, 1.7, 0],
 [2017, 60.0, 1.7, 0],
 [2006, 60.0, 1.7, 0],
 [1941, 60.0, 1.7, 7],
 [1986, 59.6, 1.3, 2],
 [2007, 59.4, 1.1, 3],
 [2005, 59.3, 1.0, 0],
 [1964, 59.3, 1.0, 1],
 [2011, 59.2, 0.9, 0],
 [1990, 59.2, 0.9, 0],
 [1987, 59.2, 0.9, 0],
 [2021, 58.9, 0.6, 0],
 [1991, 58.8, 0.5, 0],
 [1962, 58.8, 0.5, 0],
 [2020, 58.7, 0.4, 0],
 [2010, 58.7, 0.4, 0],
 [1952, 58.7, 0.4, 2],
 [2018, 58.6, 0.3, 0],
 [2015, 58.6, 0.3, 0],
 [1956, 58.5, 0.2, 1],
 [2022, 58.4, 0.1, 1],
 [2019, 58.4, 0.1, 0],
 [1994, 58.4, 0.1, 0],
 [1980, 58.4, 0.1, 0],
 [1973, 58.4, 0.1, 3],
 [2001, 58.3, 0.0, 0],
 [1995, 58.3, 0.0, 1],
 [2000, 58.2, -0.1, 0],
 [1957, 58.2, -0.1, 5],
 [2004, 58.1, -0.2, 0],
 [1984, 58.1, -0.2, 2],
 [1977, 58.1, -0.2, 0],
 [1955, 58.1, -0.2, 4],
 [1959, 58.0, -0.3, 2],
 [1992, 57.7, -0.6, 0],
 [1988, 57.7, -0.6, 0],
 [1981, 57.7, -0.6, 0],
 

In [511]:
 # maybe make a dictionary of text parameters here...

text_dict = {
    'maxt' : [
        'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS TIED FOR THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.',
        'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS TIED FOR THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.'
        ],
    'mint' : [
        'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS TIED FOR THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.',
        'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS TIED FOR THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.',
        ],
    'avgt' : [
        'THE YEARLY AVERAGE MEAN TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE MEAN TEMPERATURE OF {} ({}) WAS TIED FOR THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.',
        'THE YEARLY AVERAGE MEAN TEMPERATURE OF {} ({}) WAS THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.',
        'THE YEARLY AVERAGE MEAN TEMPERATURE OF {} ({}) WAS TIED FOR THE {} COLDEST YEAR ON RECORD, AND THE COLDEST SINCE {}.',
        ],
    'pcpn' : [
        'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS THE {} WETTEST YEAR ON RECORD AND THE WETTEST SINCE {}.',
        'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS TIED FOR THE {} WETTEST YEAR ON RECORD AND THE WETTEST SINCE {}.',
        'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS THE {} DRIEST YEAR ON RECORD AND THE DRIEST SINCE {}.',
        'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS TIED FOR THE {} DRIEST YEAR ON RECORD AND THE DRIEST SINCE {}.'
        ],
    'snow' : [
        'THE YEARLY SNOWFALL TOTAL OF {} INCHES ({}) WAS THE {} SNOWIEST YEAR ON RECORD AND THE SNOWIEST SINCE {}.', 
        'THE YEARLY SNOWFALL TOTAL OF {} INCHES ({}) WAS TIED FOR THE {} SNOWIEST YEAR ON RECORD AND THE SNOWIEST SINCE {}.', 
        'THE YEARLY SNOWFALL TOTAL OF {} INCHES ({}) WAS THE {} LEAST SNOWIEST YEAR ON RECORD AND THE LEAST SNOWIEST SINCE {}.', 
        'THE YEARLY SNOWFALL TOTAL OF {} INCHES ({}) WAS TIED FOR THE {} LEAST SNOWIEST YEAR ON RECORD AND THE LEAST SNOWIEST SINCE {}.', 
        ]    
    }


In [482]:
text_dict['maxt'][0]

'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR ON RECORD, AND THE WARMEST SINCE {}.'

In [ ]:
# here is what we had going for making the text scripts so far...

searchyear = 2023


yrly_val = [item[1] for item in maxt_filtered if item[0] == searchyear][0] # the yearly value, e.g. annual max T, min T, avg T
yrly_dfn = [item[2] for item in maxt_filtered if item[0] == searchyear][0] # the yearly value departure from normal

# if float(yrly_dfn) > 0.0:
#     yrly_dfn = f'+{yrly_dfn}'


# replacement_words = [f'{yrly_val}', f'{yrly_dfn}', f'{rank}', f'{next_closest_year}']

rank = 1
for idx, lst in enumerate(maxt_filtered):

    # check if our yearly value matches the value in the rank
    # if it does not, then keep counting the rank, if it does, then its a tie and don't count it
    if lst[1] != yrly_val:
        rank = rank + 1

    # if its our search year, then break the loop
    if lst[0] == searchyear:
        #rank = idx+1
        break

percentile = (rank/len(maxt_filtered))*100

# if the ranking is in the top 50th percentile, cut to the top half of the dataset
if percentile <= 50:
    filtered_data = maxt_filtered[:idx]

# if the ranking is in the lower 50th percentile, cut to the lower half of the dataset
elif percentile > 50:
    filtered_data = maxt_filtered[idx:]

    
# calculate residules over the filtered list to find the min residual and thus, the next closest year
res = []
for sublst in filtered_data:
    res.append(abs(searchyear - sublst[0]))

next_closest_year = filtered_data[res.index(min(res))][0]

# words to fill into the default text strings
replacement_words = [f'{yrly_val}', f'{yrly_dfn}', f'{_find_numeric_suffix(rank)}', f'{next_closest_year}']

if percentile <= 50:
    #print(f'The yearly average high temperature of {searchyear} was the {rank}th warmest year, and the warmest year since {next_closest_year}.')
    _text = text_dict['temperatures']['warmest_avghigh_since'].format(*replacement_words).upper()

elif percentile > 50:
    _text = text_dict['temperatures']['coldest_avghigh_since']

_text



In [464]:
sorted_pcpn[36:]

[[2023, 44.64, 0.14, 0],
 [1998, 44.43, -0.07, 0],
 [1992, 43.96, -0.54, 0],
 [1959, 43.82, -0.68, 0],
 [1967, 43.58, -0.92, 0],
 [1955, 43.55, -0.95, 0],
 [1994, 43.5, -1.0, 0],
 [2022, 43.23, -1.27, 1],
 [1907, 43.23, -1.27, 1],
 [1999, 43.06, -1.44, 0],
 [1918, 42.32, -2.18, 0],
 [1911, 41.93, -2.57, 0],
 [1979, 41.74, -2.76, 0],
 [2004, 41.32, -3.18, 0],
 [2014, 41.2, -3.3, 0],
 [1988, 41.19, -3.31, 0],
 [1981, 41.02, -3.48, 0],
 [2017, 40.89, -3.61, 0],
 [2000, 40.42, -4.08, 0],
 [1909, 40.18, -4.32, 0],
 [1916, 40.15, -4.35, 0],
 [2001, 40.04, -4.46, 0],
 [1966, 39.71, -4.79, 1],
 [1964, 39.41, -5.09, 0],
 [2006, 39.08, -5.42, 0],
 [1976, 38.22, -6.28, 0],
 [1912, 37.74, -6.76, 2],
 [1997, 37.59, -6.91, 0],
 [1952, 37.27, -7.23, 1],
 [1995, 36.93, -7.57, 1],
 [1932, 36.93, -7.57, 0],
 [1910, 36.71, -7.79, 0],
 [2003, 36.42, -8.08, 0],
 [1934, 36.37, -8.13, 0],
 [1972, 35.65, -8.85, 0],
 [1960, 35.58, -8.92, 1],
 [2016, 35.41, -9.09, 0],
 [1971, 35.38, -9.12, 0],
 [1980, 35.13, -9

In [ ]:
 # maybe make a dictionary of text parameters here...

text_dict = {
    'avgt' : {
        'warmest_avghigh_since' : 'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR, AND THE WARMEST SINCE {}.',
        'coldest_avghigh_since' : 'THE YEARLY AVERAGE HIGH TEMPERATURE OF {} ({}) WAS THE {} COLDEST YEAR, AND THE COLDEST SINCE {}.',

        'warmest_avglow_since' : 'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS THE {} WARMEST YEAR, AND THE WARMEST SINCE {}.',
        'coldest_avglow_since' : 'THE YEARLY AVERAGE LOW TEMPERATURE OF {} ({}) WAS THE {} COLDEST YEAR, AND THE COLDEST SINCE {}.',
        },

    'pcpn' : {
        'wettest_year_since' : 'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS THE {} WETTEST ON RECORD AND WETTEST SINCE {}.', # top 50th percentile
        'driest_year_since' : 'THE YEARLY RAINFALL TOTAL OF {} INCHES ({}) WAS THE {} DRIEST ON RECORD AND DRIEST SINCE {}.' # lower 50th percentile
        }
    

    
}


In [75]:
def _calculate_precip_annuals_and_dfn(data, key):
    
    # handle trace values inside of snow, if there are any
    data_filtered = [np.float64(val) for val in data if val not in ['T', 'M']]  # filter out trace and missing values

    # if the two lists are equal after filtering, then we filtered no T's
    if len(data) == len(data_filtered):
        ann_pcp = np.round(np.sum(data_filtered), 1)

    # if the two lists are not equal after filtering, then we filtered T's out...
    elif len(data) != len(data_filtered):
        # if the sum is still 0.0, then annual snowfall was only trace 'T'
        if np.sum(data_filtered) == 0.0:
            ann_pcp = 'T'

        # if the sum is not 0.0 after filtering out trace values, find the sum
        elif np.sum(data_filtered) > 0.0:
            ann_pcp = np.round(np.sum(data_filtered), 1)

    # find the normal value for the parameter
    annual_nrml = nrml_df[key].values[-1]
    
    # if not trace, then proceed
    if ann_pcp != 'T':
        # get the observed annual average value, and the departure from normal
        #obs_annual_avg = np.round(np.sum([float(val) for val in data]),1)
        annual_avg_dfn = np.round((ann_pcp - annual_nrml),1)
    
        return ann_pcp, annual_avg_dfn

    else:
        annual_avg_dfn = annual_nrml * -1
        return ann_pcp, annual_avg_dfn



def _calculate_temp_annuals_and_dfn(data, key):

    # find the annual normal value for the parameter
    annual_nrml = nrml_df[key].values[-1]

    # get the observed annual average value, and the departure from normal
    annual_avg = np.round(np.mean([float(val) for val in data]),1)
    annual_avg_dfn = np.round((annual_avg - annual_nrml),1)

    return annual_avg, annual_avg_dfn


In [76]:
# this will get the monthly data for a given year
year = 2023

mxt_data = [val[1] for val in data['data'] if val[0] == str(year)][0]
mnt_data = [val[2] for val in data['data'] if val[0] == str(year)][0]
avgt_data = [val[3] for val in data['data'] if val[0] == str(year)][0]

pcp_data = [val[4] for val in data['data'] if val[0] == str(year)][0]
sn_data = [val[5] for val in data['data'] if val[0] == str(year)][0]

In [77]:
# find the annual avg data and dfn's for the current year
annual_avg_mx, annual_avg_mx_dfn = _calculate_temp_annuals_and_dfn(mxt_data, key = 'MAX')
annual_avg_mn, annual_avg_mn_dfn = _calculate_temp_annuals_and_dfn(mnt_data, key = 'MIN')
annual_avg_avg, annual_avg_avg_dfn = _calculate_temp_annuals_and_dfn(avgt_data, key = 'MEAN')

annual_rain, annual_rain_dfn = _calculate_precip_annuals_and_dfn(pcp_data, key = 'PCPN')
annual_sn, annual_sn_dfn = _calculate_precip_annuals_and_dfn(sn_data, key = 'SNOW')

In [180]:
# we also need to compute the average and total values for the entirety of data...



# working here - 8/16/24
# Trying to get a list of lists with the year and annual average value, as such [yr, avg_val], while filtering out years with excessive missing data

# this is working, use this to sort annual avg/total parameters 
def _sort_annual_param(data, key, index):

    yrs = [int(lst[0]) for lst in data['data']] # strip the years
    
    # strip the parameter values (formatted as lists)
    # index value here determines which parameter is stripped
    param = [lst[index] for lst in data['data']]
    
    # convert missing to nan's
    param_filtered = [[val if val != 'M' else np.nan for val in lst] for lst in param]

    # convert values to floats, but handle trace values for when key is snow
    param_filtered = [[np.float64(val) if val != 'T' else val for val in lst] for lst in param_filtered]
    
    # get the mean values of the parameter for each year if the parameter is temp related
    if key in ['MAX', 'MIN', 'MEAN']:
        param_filtered = [np.round(np.mean(lst),1) for lst in param_filtered]

        # now lets calculate the dfn's
        dfn = [np.round(val - nrml_df[key].values[-1], 1) for val in param_filtered]

    
    elif key == 'PCPN':
        # handle trace values, though this should not come up often with rainfall data
        param_filtered = [[val if val != 'T' else 0.0 for val in lst] for lst in param_filtered]
        
        # get the annual total rainfall values
        param_filtered = [np.round(np.sum(lst),2) for lst in param_filtered]  

        # now lets calculate the dfn's
        dfn = [np.round(val - nrml_df[key].values[-1], 2) for val in param_filtered]

    
    elif key == 'SNOW':
        _param_filtered = []

        # iterate through the snowfall data to handle trace values, there will be numerous cases of trace values...
        for lst in param_filtered:
            # if there are trace values in the list, then filter them out
            if 'T' in lst:
                filtered_lst = [val if val != 'T' else 0.0005 for val in lst]
        
                # if the sum is < 0.1, then annual snowfall was only trace 'T'
                if np.sum(filtered_lst) < 0.1:
                    _param_filtered.append(0.005)
        
                # if the sum is >= 0.1 after filtering out trace values, find the sum
                elif np.sum(filtered_lst) > 0.1:
                    _param_filtered.append(np.round(np.sum(filtered_lst), 1))
        
            # if there are no trace values, then calculate sum as normal
            else:
                _param_filtered.append(np.round(np.sum(lst), 1))   

        # reclassify the temporary list as the main list again
        param_filtered = _param_filtered
        
        # now lets calculate the dfn's, handle trace values, else just normal*-1
        dfn = [np.round(val - nrml_df['SNOW'].values[-1], 1) if val != 0.005 else nrml_df['SNOW'].values[-1]*-1 for val in annual_snow_vals]     
        
    
    # now lets stitch the year, avg value, and dfn back into a list of lists
    yr_annparam_dfn = [[y, val, dfn] for y, val, dfn in zip(yrs, param_filtered, dfn)]
    
    # this will remove the nan values for sorting the data
    yr_annparam_dfn_filtered = [lst for lst in yr_annparam_dfn if not np.isnan(lst[1])]

    
    # this will create a sorted list, with the largest values first
    if key == 'MIN':
        return sorted(yr_annparam_dfn_filtered, key=itemgetter(1), reverse = False)

    else:
        return sorted(yr_annparam_dfn_filtered, key=itemgetter(1), reverse = True)


In [ ]:
sorted_avg_maxt = _sort_annual_param(data, 'MAX', 1)
sorted_avg_mint = _sort_annual_param(data, 'MIN', 2)
sorted_avgt = _sort_annual_param(data, 'MEAN', 3)

sorted_precip = _sort_annual_param(data, 'PCPN', 4)
sorted_snow = _sort_annual_param(data, 'SNOW', 5)

In [39]:
with open(f'./records/myHROrecs.txt', 'r') as f:
    recs = f.read()

In [38]:
#for line in recs.splitlines():
    if str(2024) not in line:
        return None # return a None value for recs_dict

    elif str(2024) in line:
        break # exit the for loop and continue with the function

true


In [44]:
type(recs.splitlines())

list

In [46]:
recs

'<<<< Harrison >>>>\n\nRecord High\n02/21,76 2024\n02/27,84 2024\n\nRecord Low High\n01/14,7 2024\n01/15,12 2024\n\nRecord High Low\n\nRecord Low\n01/16,-4 2024\n\nRecord Rain\n07/09,2.28 2024\n07/17,2.44 2024\n\nRecord Snow\n\n'

In [47]:
if '2024' in recs:
    print('true')

true
